In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [3]:
spark = SparkSession\
    .builder\
    .appName("InstitutionDataDictionary")\
    .getOrCreate()

In [16]:
COL_1 = StructField("NAME",StringType(), True)
COL_2 = StructField("DEV_CATG",StringType(), True)
COL_3 = StructField("DEV_NAME",StringType(), True)
COL_4 = StructField("API_DATATYP",StringType(), True)
COL_5 = StructField("INDEX",StringType(), True)
COL_6 = StructField("VAR_NAME",StringType(), True)
COL_7 = StructField("VALUE",StringType(), True)
COL_8 = StructField("LABEL",StringType(), True)
COL_9 = StructField("SOURCE",StringType(), True)
COL_10 = StructField("SHOWN_USE",StringType(), True)
COL_11 = StructField("NOTES",StringType(), True)

columnList = [COL_1, COL_2,COL_3, COL_4,COL_5, COL_6,COL_7, COL_8,COL_9,COL_10,COL_11]
InstDataDictSchema = StructType(columnList)
InstDataDictSchema

StructType(List(StructField(NAME,StringType,true),StructField(DEV_CATG,StringType,true),StructField(DEV_NAME,StringType,true),StructField(API_DATATYP,StringType,true),StructField(INDEX,StringType,true),StructField(VAR_NAME,StringType,true),StructField(VALUE,StringType,true),StructField(LABEL,StringType,true),StructField(SOURCE,StringType,true),StructField(SHOWN_USE,StringType,true),StructField(NOTES,StringType,true)))

In [17]:
InstDataDictDf = spark.read.csv(
	path='institution_data_dictionary.csv',
	header=True,
	multiLine=True,
	quote = '"',
	escape='"',
	schema=InstDataDictSchema
)

In [18]:
InstDataDictDf.show(5, truncate=True)
InstDataDictDf.printSchema()

+--------------------+--------+--------+------------+------------+--------+-----+-----+------+---------+-----+
|                NAME|DEV_CATG|DEV_NAME| API_DATATYP|       INDEX|VAR_NAME|VALUE|LABEL|SOURCE|SHOWN_USE|NOTES|
+--------------------+--------+--------+------------+------------+--------+-----+-----+------+---------+-----+
|Unit ID for insti...|    root|      id|     integer|        null|  UNITID| null| null| IPEDS|      Yes| null|
|8-digit OPE ID fo...|    root| ope8_id|      string| varchar(10)|   OPEID| null| null| IPEDS|      Yes| null|
|6-digit OPE ID fo...|    root| ope6_id|      string| varchar(10)|  OPEID6| null| null| IPEDS|      Yes| null|
|    Institution name|  school|    name|autocomplete|    fulltext|  INSTNM| null| null| IPEDS|      Yes| null|
|                City|  school|    city|autocomplete|varchar(200)|    CITY| null| null| IPEDS|      Yes| null|
+--------------------+--------+--------+------------+------------+--------+-----+-----+------+---------+-----+
o

In [19]:
InstDataDictDf.createOrReplaceTempView("InstDataDictDf")

In [22]:
spark.sql("select NAME,VAR_NAME\
          from InstDataDictDf \
          where VAR_NAME in ('UNITID','INSTNM','CITY','STABBR','ZIP',) \
			  order by name").show()

+----------------+--------+-----+-----+
|            NAME|VAR_NAME|VALUE|LABEL|
+----------------+--------+-----+-----+
|            City|    CITY| null| null|
|Institution name|  INSTNM| null| null|
+----------------+--------+-----+-----+



In [23]:
spark.sql("select NAME,VAR_NAME,VALUE,LABEL\
          from InstDataDictDf \
          where VAR_NAME in ('PREDDEG') \
			  order by name").show()

+--------------------+--------+-----+--------------+
|                NAME|VAR_NAME|VALUE|         LABEL|
+--------------------+--------+-----+--------------+
|Predominant under...| PREDDEG|    0|Not classified|
+--------------------+--------+-----+--------------+



In [24]:
spark = SparkSession\
    .builder\
    .appName("InstitutionCohorts")\
    .getOrCreate()

In [25]:
InstCohortsDf = spark.read.csv(
	path='dataset/Most-Recent-Cohorts-Institution_03142022/Most-Recent-Cohorts-Institution.csv',
	header=True,
	multiLine=True,
	quote = '"',
	escape='"'
)

In [27]:
InstCohortsDf.createOrReplaceTempView("InstCohortsDf")

In [49]:
spark.sql("select INSTNM, PREDDEG\
          from InstCohortsDf\
          where ICLEVEL = 1\
          and PREDDEG = 3").show(truncate = False)

+-----------------------------------+-------+
|INSTNM                             |PREDDEG|
+-----------------------------------+-------+
|Alabama A & M University           |3      |
|University of Alabama at Birmingham|3      |
|University of Alabama in Huntsville|3      |
|Alabama State University           |3      |
|The University of Alabama          |3      |
|Athens State University            |3      |
|Auburn University at Montgomery    |3      |
|Auburn University                  |3      |
|Birmingham-Southern College        |3      |
|South University-Montgomery        |3      |
|Faulkner University                |3      |
|Huntingdon College                 |3      |
|Heritage Christian University      |3      |
|Jacksonville State University      |3      |
|Judson College                     |3      |
|University of West Alabama         |3      |
|Miles College                      |3      |
|University of Mobile               |3      |
|University of Montevallo         

In [47]:
sparkDf = spark.sql("select INSTNM, PREDDEG\
          from InstCohortsDf\
          where ICLEVEL = 1\
          and PREDDEG = 3")

In [42]:
spark.sql("select INSTNM, PREDDEG\
          from InstCohortsDf\
          where ICLEVEL = 1\
          and PREDDEG = 3").count()

2041

In [50]:
# convert to pandas datafarme
pandasdf = sparkDf.toPandas()
pandasdf.shape

(2041, 2)

### What schools are good Schools considering test scores, diversity and family income: 
The good schools which provide good return in terms of income.Good Schools considers high SAT scores. Salary of the students will be higher in the schools which has high SAT scores.

In [62]:
spark.sql("select INSTNM,CITY,STABBR,ST_FIPS,PREDDEG,CONTROL,ADM_RATE \
          from InstCohortsDf\
          where ICLEVEL = 1\
          and PREDDEG = 3 \
          and ADM_RATE != 'NULL'\
         order by ADM_RATE DESC").show(truncate = False)

+---------------------------------------------------------------------+-------------+------+-------+-------+-------+--------+
|INSTNM                                                               |CITY         |STABBR|ST_FIPS|PREDDEG|CONTROL|ADM_RATE|
+---------------------------------------------------------------------+-------------+------+-------+-------+-------+--------+
|Saint Charles Borromeo Seminary-Overbrook                            |Wynnewood    |PA    |42     |3      |2      |1       |
|DeVry University-Missouri                                            |Kansas City  |MO    |29     |3      |3      |1       |
|Goddard College                                                      |Plainfield   |VT    |50     |3      |2      |1       |
|Bellin College                                                       |Green Bay    |WI    |55     |3      |2      |1       |
|Cleveland University-Kansas City                                     |Overland Park|KS    |20     |3      |2      |1 